In [ ]:
%cd /home/ubuntu/Qwen-Image-Edit-Angles

In [ ]:
from qwenimage.datamodels import QwenConfig
from qwenimage.finetuner import QwenLoraFinetuner
from qwenimage.foundation import QwenImageFoundation, QwenImageRegressionFoundation

foundation = QwenImageFoundation(QwenConfig(
    vae_image_size=1024 * 1024,
    regression_base_pipe_steps=4,
))

In [ ]:
finetuner = QwenLoraFinetuner(foundation, foundation.config)
finetuner.load("/data/reg-mse-pixel-lpips_005000", lora_rank=32)

In [ ]:
from qwenimage.experiments.experiments_qwen import PipeInputs

pipe_inputs = PipeInputs()
pipe_inputs[0]
from qwenimage.datamodels import QwenInputs

idx = 0
inps = QwenInputs(**pipe_inputs[idx])
inps.num_inference_steps = 4
inps.vae_image_override = 1024 * 1024
inps.latent_size_override = 1024 * 1024
finetuner.enable()
foundation.scheduler.config["base_shift"] = 2.5
foundation.scheduler.config["max_shift"] = 2.5
outs = foundation.base_pipe(inps)
outs[0]

In [ ]:
from qwenimage.sources import EditingSource


src = EditingSource(
    data_dir="/data/CrispEdit",
    total_per=10,
)

In [ ]:
from transformers.image_utils import load_image
import matplotlib.pyplot as plt
import torch

from qwenimage.datamodels import QwenInputs

SEED = 67

idx = 15_310
text, _, image = src[idx]
print(text)

inps = QwenInputs(
    prompt = text,
    image = [image],

    # prompt="Turn the man into a goblin",
    # image=[load_image("/home/ubuntu/Qwen-Image-Edit-Angles/scripts/assets/media_images_sample_images_34141_02388e50bbd59c3efb75.png")],
    
    # prompt="Turn the background into a deep forest",
    # image=[load_image("/home/ubuntu/Qwen-Image-Edit-Angles/scripts/assets/media_images_sample_images_34141_02388e50bbd59c3efb75.png")],
    
    # prompt="Add a large hot air balloon to the center of the scene",
    # image=[load_image("/home/ubuntu/Qwen-Image-Edit-Angles/scripts/assets/wand-3270.png")],
    
    # prompt="Remove the little girl",
    # image=[load_image("/home/ubuntu/Qwen-Image-Edit-Angles/scripts/assets/test_images_v1/test_image_1.jpg")],
    
    # prompt="Turn the corn into two peaches, one on each hand",
    # image=[load_image("/home/ubuntu/Qwen-Image-Edit-Angles/scripts/assets/test_images_v1/test_image_5.jpg")],
    
    # prompt="Colorize the sketch to a painting of a forest",
    # image=[load_image("/home/ubuntu/Qwen-Image-Edit-Angles/scripts/assets/hK1XxQ0li9QS9AlhZitJI.jpeg")],
    
    # prompt="Remove the furniture and show an completely empty room interior",
    # image=[load_image("/home/ubuntu/Qwen-Image-Edit-Angles/scripts/assets/471419729_122147939600372575_2367375612828096747_n.jpg")],
    num_inference_steps = 2,
    vae_image_override = 1024 * 1024,
    latent_size_override = 1024 * 1024,
)

finetuner.enable()
foundation.scheduler.config["base_shift"] = 2.0
foundation.scheduler.config["max_shift"] = 2.0
inps.generator = torch.Generator().manual_seed(SEED)
outs_distill = foundation.base_pipe(inps)

finetuner.disable()
foundation.scheduler.config["base_shift"] = 0.5
foundation.scheduler.config["max_shift"] = 0.9
inps.generator = torch.Generator().manual_seed(SEED)
outs_base_2step = foundation.base_pipe(inps)

inps.num_inference_steps = 50
inps.generator = torch.Generator().manual_seed(SEED)
outs_base_50step = foundation.base_pipe(inps)



fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Plot input image
axes[0].imshow(inps.image[0])
axes[0].set_title("Input Image")
axes[0].axis('off')

# Plot base 50 step output
axes[1].imshow(outs_base_50step[0])
axes[1].set_title("Base 50 Steps")
axes[1].axis('off')

# Plot base 2 step output
axes[2].imshow(outs_base_2step[0])
axes[2].set_title("Base 2 Steps")
axes[2].axis('off')

# # Plot distilled output
axes[3].imshow(outs_distill[0])
axes[3].set_title("Distilled")
axes[3].axis('off')

# Set the prompt as the main title
fig.suptitle(inps.prompt, fontsize=14, wrap=True)

plt.tight_layout()
plt.show()
